In [ ]:
# Clip must be 103 frames and normalized to -1/1 pixels/frame^2
# from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
# import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import os

use_acceleration = True
use_velocity = False
engarde_length = 10
engarde_length_buffer = 5

if use_acceleration == False and use_velocity == False:
  display(f'ERROR, acceleration or velocity must be used.')


try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab


In [ ]:
def load_clip_vector2():
  #Loads all Files from the Clip Folders and appends them to the clip_vector list
  touch_folder = ['Left_Touch', 'Right_Touch', 'Simul']
  clip_vector = []
  for i in range(3):
    if use_acceleration == True:
      path = r'/content/drive/My Drive/projects/fencing/Fencing Clips/' + touch_folder[i] + '/' + touch_folder[i] + '_Vector_Clips_Acceleration/'
    elif use_velocity == True:
      path = r'/content/drive/My Drive/projects/fencing/Fencing Clips/' + touch_folder[i] + '/' + touch_folder[i] + '_Vector_Clips_Speed/'
    else:
      display(f'Error, Acceleration and Velocity are both false.')
    fileList = os.listdir(path)
    for file in fileList:
      vector_data = pd.read_csv(os.path.join(path, file), header=None)
      arr = vector_data.to_numpy(dtype = np.int32)
      clip_vector.append(arr.astype('float32'))

  return(clip_vector)

In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
  # fit and evaluate a model
  verbose, epochs, batch_size = 0, 15, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
  # model.add(LSTM(100, dropout=0.5, input_shape=(n_timesteps,n_features)))
  #  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(103,4)))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	
  return accuracy

In [ ]:
def summarize_results(scores):
  # summarize scores
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
  
  return

In [ ]:
def run_experiment(repeats=10):
  # run an experiment
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)
  
  return

In [ ]:
#Loads the Clip_Vector
clip_vector = load_clip_vector2()

# display(f'The clip_vector type is {type(clip_vector)} and the length is {len(clip_vector)}.')
# display(f'The clip_vector type is {type(clip_vector[0])} and the shape is {clip_vector[0].shape}.')
# display(f'The clip_vector is:')
# display(clip_vector)

c = clip_vector

padding = np.array([0,0,0,0])

#Find the max length of arrays
num_of_arrays = len(c)

#Finds the max length of the arrays
max_length = 0
for i in range(len(c)):
  if len(c[i]) > max_length:
    max_length = len(c[i])

display(f'Max Length is:')
display(max_length)
display(f'c is of type: {type(c)}.')

#Pads the Arrays within c to the max length
for j in range(len(c)):
  for k in range(max_length - (len(c[j]))):
    c[j] = np.vstack((c[j], padding))

#Truncates the Initial Engarde Positioning
trunc = engarde_length + engarde_length_buffer
for j in range(len(c)):
  c[j] = c[j][trunc:]

#Zeros the acceleration/position values when the light turns on
for j in range(len(c)):
  for k in range(len(c[j])):
    if c[j][k][2] == 1:
      c[j][k][0] = 0
    if c[j][k][3] == 1:
      c[j][k][1] = 0 

#Combines the Arrays into array c
for i in range(len(c)):
  if i == 0:
    c_temp = c[0]
  else:
    c_temp = np.concatenate((c_temp, c[i]))
c = c_temp

#Finds the Maximum and Minimum of c
if abs(np.percentile(c, 1)) > abs(np.percentile(c, 99)):
  max_value = abs(np.percentile(c, 1))
else:
  max_value = abs(np.percentile(c, 99))

display(f'The max_value is {max_value}.')

#Normalizes the values in c and sets a maximum value of max_value
for i in range(len(c)):
  for j in range(2):
    if c[i][j] < max_value:
      c[i][j] = c[i][j] * (1/max_value)
    else:
      #Preserves the sign of the value
      c[i][j] = c[i][j]/(abs(c[i][j]))

#Reshapes c to individual arrays
c = c.reshape(num_of_arrays, max_length - trunc, 4)

display(f'The shape of the array c is {c.shape}.')


'Max Length is:'

103

"c is of type: <class 'list'>."

'The max_value is 36.0.'

'The shape of the array c is (222, 88, 4).'

In [ ]:
max_features = 3 # 0,1,2 (total of 3)

# x is the input data
x = c

#Builds y based on files in the folders
touch_folder = ['Left_Touch', 'Right_Touch', 'Simul']
y = []

for i in range(len(touch_folder)):
  if use_acceleration == True:
    directory = r'/content/drive/My Drive/projects/fencing/Fencing Clips/' + touch_folder[i] + '/' + touch_folder[i] + '_Vector_Clips_Acceleration'
  elif use_velocity == True:
    directory = r'/content/drive/My Drive/projects/fencing/Fencing Clips/' + touch_folder[i] + '/' + touch_folder[i] + '_Vector_Clips_Speed'
  else:
    display(f'Both Acceleration and Speed are False.')
  %cd $directory
  number_of_files = len(os.listdir())
  display(f'The number of files in the {touch_folder[i]} is {number_of_files}.')

  for j in range(number_of_files):
    y.append(i)

y = np.array(y,dtype=np.int32)

# Convert y2 to dummy variables
y2 = np.zeros((y.shape[0], max_features),dtype=np.float32)
y2[np.arange(y.shape[0]), y] = 1.0

#Creates Test and Train Sets
x_train, x_test, y_train, y_test = train_test_split(x, y2, test_size=0.1, shuffle= True)
# display(x_train)
# display(x_test)
# display(y_train)
# display(y_test)

# Map for readable classnames
class_names = [0, 1, 2]


print('Building model...')

# evaluate_model(x_train, y_train, x_test, y_test)


n_timesteps = c.shape[1]
n_features = c.shape[2]

model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(n_timesteps,n_features)))
model.add(Dense(3, activation='sigmoid'))

# opt = keras.optimizers.Adam(learning_rate=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# # monitor = EarlyStopping(monitor='loss', min_delta=5e-1, patience=3, verbose=1, mode='auto', restore_best_weights=True)

# x_train = x
# y_train = y2

print('Train...')
# model.fit(x,y2, batch_size = 32, epochs=20, verbose = 1, steps_per_epoch = 100)
model.fit(x,y2, epochs=1200, verbose = 1,)
pred = model.predict(x)
predict_classes = np.argmax(pred,axis=1)
print("Predicted classes:",predict_classes)
print("Expected classes:", y)

/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/Left_Touch_Vector_Clips_Acceleration


'The number of files in the Left_Touch is 94.'

/content/drive/My Drive/projects/fencing/Fencing Clips/Right_Touch/Right_Touch_Vector_Clips_Acceleration


'The number of files in the Right_Touch is 79.'

/content/drive/My Drive/projects/fencing/Fencing Clips/Simul/Simul_Vector_Clips_Acceleration


'The number of files in the Simul is 49.'

Building model...
Train...
Epoch 1/1200
7/7 [==============================] - 2s 238ms/step - loss: 1.0959 - accuracy: 0.3739
Epoch 2/1200
7/7 [==============================] - 2s 251ms/step - loss: 1.0713 - accuracy: 0.4234
Epoch 3/1200
7/7 [==============================] - 2s 245ms/step - loss: 1.0607 - accuracy: 0.4234
Epoch 4/1200
7/7 [==============================] - 2s 224ms/step - loss: 1.0682 - accuracy: 0.4234
Epoch 5/1200
7/7 [==============================] - 2s 242ms/step - loss: 1.0609 - accuracy: 0.4234
Epoch 6/1200
7/7 [==============================] - 2s 244ms/step - loss: 1.0642 - accuracy: 0.4234
Epoch 7/1200
7/7 [==============================] - 2s 259ms/step - loss: 1.0559 - accuracy: 0.4234
Epoch 8/1200
7/7 [==============================] - 2s 246ms/step - loss: 1.0590 - accuracy: 0.4234
Epoch 9/1200
7/7 [==============================] - 2s 233ms/step - loss: 1.0561 - accuracy: 0.4234
Epoch 10/1200
7/7 [==============================] - 2s 237ms/step - loss

In [ ]:
# Get Model Predictions for test data
# predicted_classes = model.predict_classes(x_valid)

# y_test = y_test.astype('int16')
# predicted_classes = predicted_classes.astype('int16')

# print(classification_report(y_valid, predicted_classes, target_names=class_names))

# print(classification_report(y_test, predicted_classes, target_names=class_names))

# # display(x_test)
# predicted_classes = predicted_classes.astype('int16')
# y_test = y_test.astype('int16') 
# display(y_test)
# display(y_test.dtype)
# # display(type(predicted_classes))
# display(predicted_classes)
# print(classification_report(y_test, predicted_classes))

In [ ]:
model.save('/content/drive/My Drive/projects/fencing/Fencing Clips/ROW_model.h5')

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
display(model.summary())

Accuracy: 100.00%
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               68096     
_________________________________________________________________
dense (Dense)                (None, 3)                 387       
Total params: 68,483
Trainable params: 68,483
Non-trainable params: 0
_________________________________________________________________


None

In [ ]:
def runit(model, inp):
    inp = np.array(inp,dtype=np.float32)
    pred = model.predict(inp)
    display(pred)
    return np.argmax(pred[0])

# #Requires entering the data from a clip
# x = [
#      [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[1,1],[1,1],[1,1],[1,1],[0,0],[0,0],[0,0]]
#      ]

# print( runit( model, x ))